In [ ]:
---
title: "REC 가격예측 연습1"
author: "youngji"
date: '2017-04-03'
---

In [ ]:
REC(Renewable Energy Certificate)은 신재생에너지 생산에 대한 대가로 주어지는 증서이고, 이것을 재생에너지를 의무적으로 생산해야 하는 발전사에게 팔수 있다. 최근 SMP단가가 하락함에 따라, REC가격은 신재생에너지 발전수익에 더욱 중요한 역할을 한다. REC가격은 16년 5월 부터 태양광/비태양광 시장이 통합되었고, 17년 3월부터 양방향 거래시장이 활성화 되었다. REC가격은 정책적인 영향을 많이 받아서 안정적인 예측이 힘들며, 최근에도 시장규칙이 변경되었기 때문에 예측하기는 더더욱 어렵다. 

본 예측에서는 선형회귀(Linear Regression)과 K-최근접 이웃법을 활용해서 REC가격을 예측해보겠다. 

In [3]:
import pandas as pd
import os
from sklearn import neighbors

os.chdir("/Users/youngji/Dropbox/data")
smp_price1=pd.read_csv("smp_price.csv",header=1,names=['육지(SMP)','제주(SMP)','SMP','BLMP'])
smp_price2=smp_price1.sort_index(axis=0, ascending=True)
smp_price3=smp_price2.set_index(keys=pd.period_range(start='2001-04',periods=smp_price2.shape[0],freq='M'))
smp_price4=smp_price3['SMP']

elec_supply1=pd.read_csv("elec_supply.csv",header=0,names=["기간","설비용량","공급능력","최대전력","공급예비력","공급예비율","기준시간"])
elec_supply2=elec_supply1.sort_values("기간", ascending=True)
elec_supply3=elec_supply2[elec_supply2.기간!='2002/08']
elec_supply4=elec_supply3.set_index(keys=pd.period_range(start='2003-01',periods=elec_supply3.shape[0],freq='M'))
elec_supply4

,기간,설비용량,공급능력,최대전력,공급예비력,공급예비율,기준시간
2003-01,2003/01,"5,380","5,248","4,505",743,17,2003/01/07(19:00)
2003-02,2003/02,"5,515","5,080","4,357",723,17,2003/02/12(24:00)
2003-03,2003/03,"5,515","5,171","4,270",901,21,2003/03/04(23:00)
2003-04,2003/04,"5,515","5,004","4,084",920,23,2003/04/09(23:00)
2003-05,2003/05,"5,562","4,729","4,068",661,16,2003/05/28(15:00)
2003-06,2003/06,"5,562","5,161","4,204",957,23,2003/06/20(15:00)
2003-07,2003/07,"5,607","5,581","4,303","1,278",30,2003/07/16(15:00)
2003-08,2003/08,"5,608","5,549","4,739",810,17,2003/08/22(12:00)
2003-09,2003/09,"5,608","5,264","4,488",776,17,2003/09/05(15:00)
2003-10,2003/10,"5,605","5,303","4,247","1,056",25,2003/10/28(19:00)
